# Exercício Avaliativo 4 - Banco de dados orientado à colunas e Cassandra
Estoque da Montadora 

Um fabricante de automóveis contratou você para desenvolver um sistema de banco de dados distribuído usando o Cassandra para as linhas de montagem de toda a corporação, onde cada máquina pudesse acessar a base de dados e buscar as peças de maneira correta para ser montada nos respectivos modelos de veículos. Para isso, você deverá criar a tabela estoque no sistema DataStax ASTRA e inserir as colunas usando o arquivo auxiliar disponibilizado junto com essa atividade. 


# Diego Anestor Coutinho
## 1692

**Banco de dados executando em ambiente local,**  
**favor conferir as instruções no arquivo README.md**

In [1]:
import json

from cassandra.cluster import Cluster
from cassandra.query import dict_factory
from cassandra.auth import PlainTextAuthProvider

auth_provider = PlainTextAuthProvider(username='admin', password='admin')
cluster = Cluster(contact_points=['127.0.0.1'], auth_provider=auth_provider)

session = cluster.connect()
session.row_factory = dict_factory

In [2]:
# linha abaixo referenciado da internet para criação de keyspace
session.execute('CREATE KEYSPACE IF NOT EXISTS montadora WITH REPLICATION = {\'class\': \'SimpleStrategy\', \'replication_factor\': 1}')
session.set_keyspace('montadora')

## Questão 1: Siga os itens listados abaixo: 

Faça a inserção de uma nova peça com os dados abaixo: 

id: 5  
nome: Pistao  
carro: Mustang  
estante: 4  
nível: 1  
quantidade: 167  

Faça a inserção de uma nova peça com os dados abaixo: 

id: 4  
nome: Suspencao   
carro: Argo   
estante: 1   
nível: 1   
quantidade: 3500   

In [3]:
# criação da tabela
session.execute('''
    CREATE TABLE IF NOT EXISTS parts (
        id int PRIMARY KEY,
        nome text,
        carro text,
        estante int,
        nivel int,
        quantidade int
    )
''')

def inserir(session, part):
    query = '''
    INSERT INTO parts (id, nome, carro, estante, nivel, quantidade)
    VALUES (%(id)s, %(nome)s, %(carro)s, %(estante)s, %(nivel)s, %(quantidade)s)
    '''
    session.execute(query, part)

In [4]:
class Part:
    def __init__(self, id, nome, carro, estante, nivel, quantidade):
        self.id = id
        self.nome = nome
        self.carro = carro
        self.estante = estante
        self.nivel = nivel
        self.quantidade = quantidade

    def dict(self):
        return {
            'id': self.id,
            'nome': self.nome,
            'carro': self.carro,
            'estante': self.estante,
            'nivel': self.nivel,
            'quantidade': self.quantidade
        }

In [5]:
part1 = Part(5, 'Pistao', 'Mustang', 4, 1, 167)
part2 = Part(4, 'Suspencao', 'Argo', 1, 1, 3500)

# inserção dos dados
inserir(session, part1.dict())
inserir(session, part2.dict())

## Questão 2: Escreva o comando CQL utilizado para cada item abaixo: 

- Faça uma busca no banco de dados que retorno todos os dados do item com nome 'Pistão';
- Faça uma busca no banco que calcule a média aritmética da quantidade de todas as colunas armazenadas na tabela;
- Faça uma busca que retorne quantas colunas tem armazenadas na tabela;
- Busque a maior e a menor quantidade de peças usando as alias "maior quantidade" e "menor quantidade" para a tabela estoque;
- Faça uma busca que retorne os atributos nome, carro e quantidade, onde a estante seja igual a 3;
- Faça uma busca que retorne a média aritmética da quantidade onde o nível seja igual a 1; 
- Faça uma busca retornando todos os atributos onde a estante seja menor do que 3 e o nível seja maior do que 4;
 

In [6]:
# Faça uma busca no banco de dados que retorno todos os dados do item com nome 'Pistão'
rows = session.execute("SELECT * FROM parts WHERE nome = 'Pistao' ALLOW FILTERING")
list(rows)

[{'id': 5,
  'carro': 'Mustang',
  'estante': 4,
  'nivel': 1,
  'nome': 'Pistao',
  'quantidade': 167}]

In [7]:
# Faça uma busca no banco que calcule a média aritmética da quantidade de todas as colunas armazenadas na tabela
rows = session.execute("SELECT AVG(quantidade) FROM parts")
list(rows)

[{'system.avg(quantidade)': 1833}]

In [8]:
# Faça uma busca que retorne quantas colunas tem armazenadas na tabela
rows = session.execute("SELECT COUNT(*) FROM parts")
list(rows)

[{'count': 2}]

In [9]:
# Busque a maior e a menor quantidade de peças usando as alias "maior quantidade" e "menor quantidade" para a tabela estoque
rows = session.execute("SELECT MAX(quantidade) AS maior_quantidade, MIN(quantidade) AS menor_quantidade FROM parts")
list(rows)

[{'maior_quantidade': 3500, 'menor_quantidade': 167}]

In [10]:
# Faça uma busca que retorne os atributos nome, carro e quantidade, onde a estante seja igual a 3
rows = session.execute("SELECT nome, carro, quantidade FROM parts WHERE estante = 3 ALLOW FILTERING")
list(rows)

[]

In [11]:
# Faça uma busca que retorne a média aritmética da quantidade onde o nível seja igual a 1
rows = session.execute("SELECT AVG(quantidade) FROM parts WHERE nivel = 1 ALLOW FILTERING")
list(rows)

[{'system.avg(quantidade)': 1833}]

In [12]:
# Faça uma busca retornando todos os atributos onde a estante seja menor do que 3 e o nível seja maior do que 4
rows = session.execute("SELECT * FROM parts WHERE estante < 3 AND nivel > 4 ALLOW FILTERING")
list(rows)

[]

## Questão 3: Elabore um script Python que seja capaz de fazer uma consulta mostrando: 
nome, estante e quantidade do carro fornecido pelo usuário. 

In [13]:
carro = input('Digite o tipo do carro: ') # input: Mustang
rows = session.execute(f"SELECT nome, estante, quantidade FROM parts WHERE carro = '{carro}' ALLOW FILTERING")
list(rows)

[{'nome': 'Pistao', 'estante': 4, 'quantidade': 167}]